In [ ]:
!pip install --upgrade huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.5/450.5 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7


In [ ]:
import os
from huggingface_hub import login

# login(token="") #read_token
login(token="") #write_token

In [ ]:
!pip install --upgrade transformers peft accelerate datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 82.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.34.2
    Uninstalling accelerate-0.34.2:
      Successfully uninstalled accelerate-0.34.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [ ]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 25.5 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    BitsAndBytesConfig,
    DataCollatorWithPadding
)
from torch.optim.lr_scheduler import ReduceLROnPlateau
from peft import LoraConfig, get_peft_model
import numpy as np
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from transformers import get_cosine_schedule_with_warmup

# Load dataset
dataset = load_dataset("ka05ar/Det_ai_mala_text")


# Preprocess labels


from datasets import DatasetDict

# Define the label mapping function
def preprocess_labels(example):
    label_mapping = {"HUMAN": 0, "AI": 1}
    if "LABEL" in example:
        if example["LABEL"] in label_mapping:
            example["labels"] = label_mapping[example["LABEL"]]
        else:
            raise ValueError(f"Unexpected label: {example['LABEL']}")
    else:
        raise KeyError("'LABEL' key not found in the dataset example.")
    return example

# Apply preprocessing to the dataset
processed_dataset = dataset.map(preprocess_labels)

# Split the train and test sets from the processed dataset
split_dataset = DatasetDict({
    "train": processed_dataset["train"].train_test_split(test_size=0.2, seed=42)["train"],
    "test": processed_dataset["train"].train_test_split(test_size=0.2, seed=42)["test"]
})

# Access the train and test datasets
train_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]

# Verify the processed dataset
print(f"First example in train dataset: {train_dataset[0]}")
print(f"First example in test dataset: {test_dataset[0]}")



# def preprocess_labels(example):
#     label_mapping = {"HUMAN": 0, "AI": 1}
#     example["labels"] = label_mapping[example["LABEL"]]
#     return example


# processed_dataset = dataset.map(preprocess_labels)
# split_dataset = processed_dataset.train_test_split(test_size=0.2, seed=42)
# train_dataset = processed_dataset["train"]
# test_dataset = processed_dataset["validation"]


hum_ai_mala_train.csv:   0%|          | 0.00/247k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

First example in train dataset: {'ID': 'MAL_HUAI_TR_186', 'DATA': '8 ലക്ഷതിന് ഈ വാഹനം കിട്ടൂല റോഡിൽ ഇറക്കാൻ 10 ലക്ഷം ആവും ഈ വിലയിൽ വങ്ങാൻ കിട്ടുന്ന അടിപൊളി വാഹനം', 'LABEL': 'HUMAN', 'labels': 0}
First example in test dataset: {'ID': 'MAL_HUAI_TR_097', 'DATA': 'ഇന്റീരിയർ കൂടി ചേഞ്ച്\u200c ആകാമായിരുന്നു കണ്ട് മടുത്ത ഡിസൈൻ', 'LABEL': 'HUMAN', 'labels': 0}


In [ ]:
# Initialize tokenizer with proper padding setup
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b")
tokenizer.pad_token = tokenizer.eos_token  # Use EOS token as PAD token
tokenizer.padding_side = 'right'

# Preprocessing function
def preprocess_function(examples):
    tokenized = tokenizer(
        examples["DATA"],
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors=None
    )
    tokenized['labels'] = examples['labels']
    return tokenized

# Encode datasets
encoded_train = train_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=train_dataset.column_names
)
encoded_test = test_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=test_dataset.column_names
)

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Map:   0%|          | 0/640 [00:00<?, ? examples/s]

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

In [ ]:
# Set up model
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float32
)

# Initialize model with proper padding token configuration
model = AutoModelForSequenceClassification.from_pretrained(
    "google/gemma-2-2b",
    quantization_config=bnb_config,
    device_map="auto",
    num_labels=2,
    torch_dtype=torch.float32,
    pad_token_id=tokenizer.pad_token_id,  # Set pad_token_id to eos_token_id
)

# Configure model padding token
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False  # Disable cache to prevent memory issues

# Configure LoRA
lora_config = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS"
)

model = get_peft_model(model, lora_config)


config.json:   0%|          | 0.00/818 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/481M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some weights of Gemma2ForSequenceClassification were not initialized from the model checkpoint at google/gemma-2-2b and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Prepare data loaders

from torch.utils.data import WeightedRandomSampler

def get_balanced_sampler(dataset):
    labels = [example['labels'] for example in dataset]
    class_counts = np.bincount(labels)
    weights = 1. / torch.tensor(class_counts, dtype=torch.float)
    sample_weights = weights[labels]

    sampler = WeightedRandomSampler(
        weights=sample_weights,
        num_samples=len(sample_weights),
        replacement=True
    )
    return sampler

# Use in DataLoader
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
train_dataloader = DataLoader(
    encoded_train,
    sampler=get_balanced_sampler(encoded_train),
    batch_size=16,
    collate_fn=data_collator
)

eval_dataloader = DataLoader(
    encoded_test,
    batch_size=16,
    collate_fn=data_collator
)

In [ ]:
# Training configuration
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=1e-4,
    weight_decay=0.01,
    eps=1e-8,
    betas=(0.9, 0.999)
)

# Add learning rate scheduler
num_epochs = 10
num_training_steps = num_epochs * len(train_dataloader)

# Calculate warmup steps (10% of total steps)
# num_warmup_steps = int(0.1 * num_training_steps)

# Create scheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau

scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)


In [ ]:
# Training loop
device = "cuda" if torch.cuda.is_available() else "cpu"
progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    total_loss = 0
    for batch_idx, batch in enumerate(train_dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)
        loss = outputs.loss

        if torch.isnan(loss).any():
            print(f"NaN loss detected at batch {batch_idx}!")
            continue

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()
        progress_bar.update(1)

        if (batch_idx + 1) % 100 == 0:
            avg_loss = total_loss / (batch_idx + 1)
            print(f"Epoch {epoch+1}, Batch {batch_idx+1}, Average Loss: {avg_loss:.4f}")

    # Evaluation
    model.eval()
    eval_loss = 0
    predictions = []
    references = []

    with torch.no_grad():
        for batch in eval_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)

            eval_loss += outputs.loss.item()
            predictions.extend(torch.argmax(outputs.logits, dim=-1).cpu().numpy())
            references.extend(batch["labels"].cpu().numpy())

    eval_loss /= len(eval_dataloader)
    accuracy = np.mean(np.array(predictions) == np.array(references))

    scheduler.step(eval_loss)

    print(f"\nEpoch {epoch+1}:")
    print(f"Average Training Loss: {total_loss/len(train_dataloader):.4f}")
    print(f"Evaluation Loss: {eval_loss:.4f}")
    print(f"Accuracy: {accuracy:.4f}")

  0%|          | 0/400 [00:00<?, ?it/s]


Epoch 1:
Average Training Loss: 0.5237
Evaluation Loss: 0.3458
Accuracy: 0.9250

Epoch 2:
Average Training Loss: 0.1649
Evaluation Loss: 0.3480
Accuracy: 0.9500

Epoch 3:
Average Training Loss: 0.0648
Evaluation Loss: 0.3428
Accuracy: 0.9437

Epoch 4:
Average Training Loss: 0.0366
Evaluation Loss: 0.3444
Accuracy: 0.9563

Epoch 5:
Average Training Loss: 0.0016
Evaluation Loss: 0.3683
Accuracy: 0.9625

Epoch 6:
Average Training Loss: 0.0000
Evaluation Loss: 0.3798
Accuracy: 0.9688

Epoch 7:
Average Training Loss: 0.0228
Evaluation Loss: 0.3647
Accuracy: 0.9625

Epoch 8:
Average Training Loss: 0.0000
Evaluation Loss: 0.3640
Accuracy: 0.9625

Epoch 9:
Average Training Loss: 0.0000
Evaluation Loss: 0.3746
Accuracy: 0.9688

Epoch 10:
Average Training Loss: 0.0000
Evaluation Loss: 0.3752
Accuracy: 0.9688


In [ ]:
# Save the model
model.save_pretrained("AI_Mala_google_gemma_final_model_updated2")
tokenizer.save_pretrained("AI_Mala_google_gemma_final_model_updated2")

('AI_Mala_google_gemma_final_model_updated2/tokenizer_config.json',
 'AI_Mala_google_gemma_final_model_updated2/special_tokens_map.json',
 'AI_Mala_google_gemma_final_model_updated2/tokenizer.model',
 'AI_Mala_google_gemma_final_model_updated2/added_tokens.json',
 'AI_Mala_google_gemma_final_model_updated2/tokenizer.json')

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the locally saved model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("AI_Mala_google_gemma_final_model_updated2")
tokenizer = AutoTokenizer.from_pretrained("AI_Mala_google_gemma_final_model_updated2")

# Push to Hugging Face Hub
model.push_to_hub("ka05ar/AI_Mala_google_gemma_test_updated2")
tokenizer.push_to_hub("ka05ar/AI_Mala_google_gemma_test_updated2")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some weights of Gemma2ForSequenceClassification were not initialized from the model checkpoint at google/gemma-2-2b and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


adapter_model.safetensors:   0%|          | 0.00/3.23M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ka05ar/AI_Mala_google_gemma_test_updated2/commit/8586acb7793c5a6895b429663fef469165b69633', commit_message='Upload tokenizer', commit_description='', oid='8586acb7793c5a6895b429663fef469165b69633', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ka05ar/AI_Mala_google_gemma_test_updated2', endpoint='https://huggingface.co', repo_type='model', repo_id='ka05ar/AI_Mala_google_gemma_test_updated2'), pr_revision=None, pr_num=None)